### RAG(Retrieval Augmented Generation) practice 
- Retrieval -Find relevant information given a query
- Augmentation -Take the relevant information from retrieval and augment our input(prompt) to an LLM with that relevant information.
- Generation - Take the first two steps and pass them to an LLM for generative outputs.

### Why RAG?
- The main goal of RAG is to improve the generation outputs of LLMs. 
- Most of what I practiced were based on fine-tuning. It can provide up-to-date information to the model with less efforts.
### Steps
- Import PDF
- Process text for embedding
- Embed text chunks with embedding model.
- Save embeddings to file for later
### Practiced with 
- Daniel Bourke- Local Retrieval Generation(RAG) from Scratch(step by step tutorial)
- https://www.youtube.com/watch?v=qN_2fnOPY-M&t=513s

### Import PDF Document

In [ ]:
import os 
import requests

# Get PDF docu path
pdf_path = "ECE Graduate Policy Manual AY2023-2024"

# download 
if not os.path.exist(pdf_path):
    print(f"[INFO] File doesn't exist, downloading...")

    # Enter the URL of the PDF 
    # VT ECE graduate policy link 
    url = "example_url" 

    filename = pdf_path 